In [1]:
import os
import sys

In [2]:
os.chdir('/home/au643300/DataHandling/')
sys.path.append('/home/au643300/DataHandling/')

In [3]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, as_completed
import glob
from src.data.tonetCDF import to_netcdf
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from src.interim import read_valdata
import numpy as np

In [5]:
cluster=SLURMCluster(cores=8,
                     processes=1,
                     memory="62GB",
                     queue='q64',
                     walltime='0-00:30:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     extra=['--resources mem=15']
                    )
                    

In [7]:
cluster.adapt(minimum_jobs=1,maximum_jobs=16)

In [6]:
client=Client(cluster)

Her skal jeg bruge xr.open_mfdataset sammen med glob til at lave en liste over alle nc filerne, åbne dem op og tjekke om coordinaterne matcher en reference liste, list(ds.keys())